In [1]:
import pandas as pd
import re

In [2]:
# ZHVI All Homes (SFR, Condo/Co-op) Time Series ($) (by ZIP code)
csv_home_values = "Resources/Zip_Zhvi_AllHomes.csv"

# Median Sale Price - Seasonally Adjusted ($) (by ZIP code)
csv_home_sales = "Resources/Sale_Prices_Zip.csv"

# Monthly Home Sales (Number, Raw) (by ZIP code)
csv_number_sales = "Resources/Sale_Counts_Zip.csv"

# Foreclosure Resales (%) (by ZIP code)
csv_foreclosure_resales = "Resources/SalesPrevForeclosed_Share_Zip.csv"

home_values = pd.read_csv(csv_home_values, encoding="ISO-8859-1")
home_sales = pd.read_csv(csv_home_sales, encoding="ISO-8859-1")
number_sales = pd.read_csv(csv_number_sales, encoding="ISO-8859-1")
foreclosure_resales = pd.read_csv(csv_foreclosure_resales, encoding="ISO-8859-1")

# Suffixes:
# ZVHI = Zillow Home Value Index ($)
# MSR = Median Sales Price - Seasonally Adjusted ($)
# NS = Number of Sales
# FR = Foreclosure Resales (%)




In [42]:
# Python Dictionary to translate US States to Two letter codes
# Dictionary obtained from rogerallen (https://gist.github.com/rogerallen/1583593)
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

# Rename StateName columns
home_values = home_values.rename(columns={"RegionName":"ZIP Code",
                                          "2008-01":"2008-01_ZHVI",
                                          "2008-02":"2008-02_ZHVI"})
home_sales = home_sales.rename(columns={"RegionName":"ZIP Code",
                                        "StateName":"State"})
number_sales = number_sales.rename(columns={"RegionName":"ZIP Code",
                                            "StateName":"State",
                                            "seasAdj":"seasAdj_NS"})
foreclosure_resales = foreclosure_resales.rename(columns={"RegionName":"ZIP Code",
                                                          "StateName":"State"})

# Translate states into two letter codes
try:
    for index, row in home_sales.iterrows():
        state = home_sales.loc[index, "State"]
        abbr = us_state_abbrev[state]
        home_sales.loc[index, "State"] = abbr

    
    for index, row in number_sales.iterrows():
        state = number_sales.loc[index, "State"]
        abbr = us_state_abbrev[state]
        number_sales.loc[index, "State"] = abbr

    for index, row in foreclosure_resales.iterrows():
        state = foreclosure_resales.loc[index, "State"]
        abbr = us_state_abbrev[state]
        foreclosure_resales.loc[index, "State"] = abbr
except:
    pass

home_values.head()

,RegionID,ZIP Code,City,State,Metro,CountyName,SizeRank,2008-01_ZHVI,2008-02_ZHVI,2008-03,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,1,NaN,NaN,NaN,...,4.846133e+05,4.844847e+05,4.856167e+05,4.865600e+05,4.858847e+05,4.849017e+05,4.823117e+05,4.800917e+05,4.787937e+05,4.785953e+05
1,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,2,NaN,NaN,NaN,...,1.192562e+06,1.184811e+06,1.182110e+06,1.185962e+06,1.188839e+06,1.181644e+06,1.163996e+06,1.148232e+06,1.148030e+06,1.163213e+06
2,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,3,269106.0,268326.0,267699.333333,...,3.352377e+05,3.350893e+05,3.339180e+05,3.331563e+05,3.325493e+05,3.327513e+05,3.328630e+05,3.339837e+05,3.346290e+05,3.352343e+05
3,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,NaN,NaN,NaN,...,6.303370e+05,6.304280e+05,6.328107e+05,6.339393e+05,6.326857e+05,6.306867e+05,6.281890e+05,6.261287e+05,6.251030e+05,6.240497e+05
4,91940,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,5,128064.0,127776.0,127537.666667,...,1.814760e+05,1.818737e+05,1.817140e+05,1.818840e+05,1.823700e+05,1.831330e+05,1.839323e+05,1.843723e+05,1.851663e+05,1.863170e+05


In [6]:
merge1 = pd.merge(home_values, home_sales, on=["ZIP Code","State","RegionID"], how="outer", suffixes=("_ZHVI", "_MSR"))
merge2 = pd.merge(number_sales, foreclosure_resales, on=["ZIP Code","State","RegionID"], how="outer", suffixes=("_NS", "_FR"))
main_data = pd.merge(merge1, merge2, on=["ZIP Code","State","RegionID"], how="outer")

In [7]:
main_data.head()

,RegionID,ZIP Code,City,State,Metro,CountyName,SizeRank_ZHVI,2008-01_ZHVI,2008-02_ZHVI,2008-03_ZHVI,...,2019-02_FR,2019-03_FR,2019-04_FR,2019-05_FR,2019-06_FR,2019-07_FR,2019-08_FR,2019-09_FR,2019-10_FR,2019-11_FR
0,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,1.0,NaN,NaN,NaN,...,0.0000,0.0092,0.0064,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,3.0,269106.0,268326.0,267699.333333,...,0.0058,0.0083,0.0072,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4.0,NaN,NaN,NaN,...,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,91940,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,5.0,128064.0,127776.0,127537.666667,...,0.0000,0.0123,0.0123,0.0181,0.006,NaN,NaN,NaN,NaN,NaN


In [63]:
def findavg(data,year,suffix):
    x = []
    for i in data.columns:
        y = (re.findall(f'^{year}-([0-9][0-9])_{suffix}',i))
        [x.append(int(i)) for i in y]
    min_col = data.columns.get_loc(f"{year}-{min(x):02d}_{suffix}")
    max_col = data.columns.get_loc(f"{year}-{max(x):02d}_{suffix}")
    data[f"{year}_Avg_{suffix}"] = data.iloc[:,min_col:max_col+1].mean(axis=1)
    return data[f"{year}_AVG_{suffix}"]

In [59]:
new_data = main_data[['RegionID', 'ZIP Code', 'City', 'State', 'CountyName']]
new_data.head()

,RegionID,ZIP Code,City,State,CountyName
0,84654,60657,Chicago,IL,Cook County
1,61637,10023,New York,NY,New York County
2,91982,77494,Katy,TX,Harris County
3,84616,60614,Chicago,IL,Cook County
4,91940,77449,Katy,TX,Harris County


In [60]:
years = [i for i in range(2008,2020,1)]
suffixes = ["ZHVI","MSR","NS","FR"]

for suffix in suffixes:
    for year in years:
        new_data = new_data.join(findavg(main_data,year,suffix))


In [62]:
new_data.head(20)

,RegionID,ZIP Code,City,State,CountyName,2008_Avg_ZHVI,2009_Avg_ZHVI,2010_Avg_ZHVI,2011_Avg_ZHVI,2012_Avg_ZHVI,...,2010_Avg_FR,2011_Avg_FR,2012_Avg_FR,2013_Avg_FR,2014_Avg_FR,2015_Avg_FR,2016_Avg_FR,2017_Avg_FR,2018_Avg_FR,2019_Avg_FR
0,84654,60657,Chicago,IL,Cook County,479900.645833,466041.694444,445775.916667,423604.722222,412033.055556,...,0.000633,0.000550,0.001392,0.025442,0.016950,0.003683,0.005017,0.004208,0.005367,0.006700
1,61637,10023,New York,NY,New York County,NaN,833515.733333,836375.277778,858051.166667,888688.861111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,91982,77494,Katy,TX,Harris County,267059.388889,267583.416667,274168.222222,266745.861111,271493.027778,...,0.038658,0.029625,0.029517,0.010633,0.004033,0.002458,0.003492,0.002967,0.001367,0.007025
3,84616,60614,Chicago,IL,Cook County,606633.729167,594972.111111,565302.194444,534765.361111,523258.083333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002430,0.001183,0.006750
4,91940,77449,Katy,TX,Harris County,125067.777778,119876.111111,119721.722222,112142.083333,110622.833333,...,0.235167,0.206533,0.189350,0.055933,0.031283,0.013625,0.013442,0.012083,0.003492,0.011450
5,91733,77084,Houston,TX,Harris County,125401.111111,122618.666667,122123.472222,115281.500000,114020.666667,...,0.126192,0.131583,0.147692,0.048142,0.026567,0.013533,0.013725,0.006217,0.009217,0.009725
6,93144,79936,El Paso,TX,El Paso County,127441.750000,122583.194444,122555.694444,119277.388889,117206.638889,...,0.037650,0.032367,0.032308,0.028317,0.029208,0.044042,0.047142,0.022800,0.022708,0.028440
7,84640,60640,Chicago,IL,Cook County,306105.092593,295462.111111,284472.722222,262641.250000,245682.638889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006057,0.006250,0.016625
8,62037,11226,New York,NY,Kings County,NaN,NaN,518957.216667,485918.472222,487198.777778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,61807,10467,New York,NY,Bronx County,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
state_data =new_data[
                     ['State', '2008_Avg_ZHVI','2009_Avg_ZHVI', '2010_Avg_ZHVI', '2011_Avg_ZHVI',
                      '2012_Avg_ZHVI', '2013_Avg_ZHVI', '2014_Avg_ZHVI', '2015_Avg_ZHVI', '2016_Avg_ZHVI',
                      '2017_Avg_ZHVI', '2018_Avg_ZHVI', '2019_Avg_ZHVI', '2008_Avg_MSR','2009_Avg_MSR', 
                      '2010_Avg_MSR', '2011_Avg_MSR', '2012_Avg_MSR','2013_Avg_MSR', '2014_Avg_MSR', 
                      '2015_Avg_MSR', '2016_Avg_MSR','2017_Avg_MSR', '2018_Avg_MSR', '2019_Avg_MSR', 
                      '2008_Avg_NS','2009_Avg_NS', '2010_Avg_NS', '2011_Avg_NS', '2012_Avg_NS','2013_Avg_NS', 
                      '2014_Avg_NS', '2015_Avg_NS', '2016_Avg_NS','2017_Avg_NS', '2018_Avg_NS', '2019_Avg_NS', 
                      '2008_Avg_FR', '2009_Avg_FR', '2010_Avg_FR', '2011_Avg_FR', '2012_Avg_FR','2013_Avg_FR', 
                      '2014_Avg_FR', '2015_Avg_FR', '2016_Avg_FR','2017_Avg_FR', '2018_Avg_FR', '2019_Avg_FR']
                    ].groupby("State").mean()
state_data

,2008_Avg_ZHVI,2009_Avg_ZHVI,2010_Avg_ZHVI,2011_Avg_ZHVI,2012_Avg_ZHVI,2013_Avg_ZHVI,2014_Avg_ZHVI,2015_Avg_ZHVI,2016_Avg_ZHVI,2017_Avg_ZHVI,...,2010_Avg_FR,2011_Avg_FR,2012_Avg_FR,2013_Avg_FR,2014_Avg_FR,2015_Avg_FR,2016_Avg_FR,2017_Avg_FR,2018_Avg_FR,2019_Avg_FR
State,,,,,,,,,,,,,,,,,,,,,
AK,257757.731674,255277.256173,255265.845679,252652.662037,257154.490616,261483.093559,260587.859272,263911.006225,260905.361420,265698.318409,...,0.017953,0.015444,0.013250,0.015368,0.010808,0.014436,0.007723,0.008110,0.008051,0.015248
AL,142287.102771,137912.212703,132560.954521,127438.048756,124757.877755,124216.831304,123857.456570,126152.310004,126710.281543,129905.843310,...,0.028620,0.024541,0.022358,0.036341,0.038174,0.036396,0.029826,0.024794,0.021197,0.018580
AR,109428.506791,105079.607528,102444.345622,100620.504960,100541.646575,101893.995841,102221.952317,102801.309430,104403.715475,106864.871102,...,0.034028,0.024479,0.011072,0.017768,0.024253,0.012681,0.012304,0.011055,0.011784,0.012021
AZ,257791.880246,222270.168886,208525.831265,192496.057547,192090.256004,210370.328371,219667.363194,224545.767161,235508.485779,245929.407627,...,0.138657,0.132389,0.072265,0.059775,0.046469,0.031105,0.021606,0.016913,0.011268,0.009412
CA,468233.711450,423004.034608,422985.651603,401367.360069,406446.894985,446805.162324,476875.116913,503387.102992,536483.483562,569431.051039,...,0.212939,0.205428,0.145055,0.065362,0.036880,0.028936,0.022030,0.013541,0.009642,0.006600
CO,253618.981200,245395.782154,241199.468952,237452.875291,239758.262169,255112.318367,270234.277054,292758.443008,313756.488091,331518.951534,...,0.088625,0.084349,0.081781,0.053862,0.028772,0.019262,0.013243,0.007637,0.002982,0.000941
CT,347249.215342,323905.392458,313118.014583,300227.801698,288950.195595,293717.917015,294941.322222,295011.945601,298319.426761,301334.038597,...,0.047259,0.038214,0.027868,0.018429,0.007117,0.037206,0.035546,0.036021,0.039914,0.031048
DC,483599.190278,458940.069444,459613.419444,459278.670833,467900.161111,506845.480159,544902.232804,563037.734127,584612.025132,609986.671958,...,0.054602,0.042308,0.007951,0.000478,0.000884,0.003264,0.003761,0.012849,0.007908,0.007087
DE,300988.790720,281541.648359,272534.585859,256029.804924,248251.028307,249140.889406,258817.413043,263802.457729,270780.876208,275315.405193,...,0.025508,0.039847,0.055848,0.040872,0.041620,0.023571,0.034314,0.045582,0.057123,0.042223
